  Copyright(c) 2021 UNIADEX, Ltd. All Rights Reserved.  
  CONFIDENTIAL  
  Author: UNIADEX, Ltd.

# 分析
切り出したショットデータを取得し、分析します。  
以下は特定ショットのデータを取得/確認し、特徴点（荷重開始点/最大荷重点/破断点）を抽出、その後全ショットから特徴点を抽出するサンプルコードです。抽出した特徴点はElasticsearchに格納されます。

In [1]:
# ライブラリインポート
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../src/'))
from data_reader.data_reader import DataReader
from elastic_manager.elastic_manager import ElasticManager
sys.path.append(os.path.join(os.getcwd(), '../src/analyzer/'))
from analyzer.h_one_extract_features import *
from analyzer.analyzer import Analyzer

Loading BokehJS ...

In [2]:
# ログ設定
from logger.logger import init_logger, get_logger

module_name = "analyzer"
init_logger(module_name)
logger = get_logger(module_name)

In [3]:
# ショットデータインデックス一覧
ElasticManager.show_indices(index="shots-*-data")

,index,docs.count,store.size
0,shots-20190523094129-data,3150000,473656
1,shots-20210101000000-data,9863,1862
2,shots-20210327141514-data,13590838,1918066
3,shots-20210708113000-data,6000,1000


In [4]:
# 対象データ設定
target = "20210327141514"
shots_data_index = "shots-" + target + "-data"
shots_meta_index = "shots-" + target + "-meta"

# 1ショットで特徴点抽出

## ショット1番を取得して確認

In [5]:
dr = DataReader()

shot_number = 50
shot_df = dr.read_shot(shots_data_index, shot_number=shot_number)
shot_df.head()

,timestamp,sequential_number,sequential_number_by_shot,rawdata_sequential_number,displacement,load01,load02,load03,load04,shot_number,tags
0,2021-03-27T05:22:44.615197,459350,0,46398509,51.79125,0.021,0.083,0.042,0.123,50,[]
1,2021-03-27T05:22:44.615207,459351,1,46398510,51.79125,0.016,0.087,0.031,0.123,50,[]
2,2021-03-27T05:22:44.615217,459352,2,46398511,51.79125,0.014,0.089,0.011,0.117,50,[]
3,2021-03-27T05:22:44.615227,459353,3,46398512,51.78250,0.013,0.083,-0.017,0.101,50,[]
4,2021-03-27T05:22:44.615237,459354,4,46398513,51.78250,0.013,0.070,-0.044,0.083,50,[]


In [6]:
# ショットメタデータを確認
shots_meta_df = dr.read_shots_meta(shots_meta_index)
shots_meta_df.head()

,timestamp,shot_number,spm,num_of_samples_in_cut_out
0,2021-03-27T05:16:25.150651,1,50.847458,109404
1,2021-03-27T05:16:26.333224,2,NaN,16564
2,2021-03-27T05:16:40.974641,3,28.571429,9877
3,2021-03-27T05:16:43.071635,4,32.608696,8640
4,2021-03-27T05:16:44.916517,5,35.928144,7798


In [7]:
# ショットの数を確認
print(f"ショット数：{len(shots_meta_df)}")

ショット数：3264


In [ ]:
# 80spmを超えるような不正なショットがないか確認
pd.set_option('display.max_rows', 3300)
shots_meta_df[shots_meta_df.spm > 80]

In [ ]:
# spm
spm = float(shots_meta_df[shots_meta_df.shot_number == 1].spm)
print(f"spm：{spm}")

## 特徴点抽出
1ショットから最大荷重点、荷重開始点、破断点を取得します。  

***shot_data***: 対象ショットデータ  
***spm***: spm  
***func***：適用するロジック  
***sub_func***：サブロジック  
***disp_chart***: グラフ表示

In [ ]:
# 最初のショットの最大荷重点、荷重開始点、破断点を確認
max_load_indicies, max_load_values, debug_values = extract_features(shot_data=shot_df, spm=spm, func=max_load, shot=shot_number, disp_chart=True)
load_start_indicies, load_start_values, debug_values = extract_features(shot_df, spm, load_start3, shot=shot_number, disp_chart=True)
break_load_indicies, break_load_values, debug_values = extract_features(shot_df, spm, breaking_vmin_amin, sub_func=narrowing_v4min_mab, shot=shot_number, disp_chart=True)

In [ ]:
# 各特徴点のインデックス番号と値を確認
print(f"【最大荷重点】\n indicies：{max_load_indicies}, values：{max_load_values}")
print(f"【荷重開始点】\n indicies：{load_start_indicies}, values：{load_start_values}")
print(f"【破断点】　　\n indicies：{break_load_indicies}, values：{break_load_values}")

# 全ショットから特徴点抽出
全ショットから最大荷重点、荷重開始点、破断点を抽出し、Elasticsearchに保存します。  

***target***: 対象ショットデータ（yyyyMMddHHMMSS）  
***shots_df***: ショットデータのDataFrame
***shots_meta_df***: ショットメタデータのDataFrame  
***feature***: 抽出する特徴点名（max/start/break）  
***func***：適用するロジック  
***sub_func***：サブロジック  
***exclude_shots***: 除外ショット(ex: exclude_shots=(1227, 1228, 1229))  

保存先インデックスは以下になります。  
* 最大荷重点：shots-yyyyMMddHHMMSS-max-point
* 荷重開始点：shots-yyyyMMddHHMMSS-start-point
* 破断点：shots-yyyyMMddHHMMSS-break-point

In [8]:
# 対象指定
shots_data_index = "shots-" + target + "-data"
shots_meta_index = "shots-" + target + "-meta"

In [9]:
%%time
dr = DataReader()
shots_df = dr.multi_process_read_all(shots_data_index)

2021-07-19 13:45:29 [INFO] データを全件取得します。データ件数が多い場合、長時間かかる場合があります。
2021-07-19 13:45:29 [INFO] Data read start. data_count: 1000000.
2021-07-19 13:45:45 [INFO] Data read start. data_count: 1000000.
2021-07-19 13:46:04 [INFO] Data read start. data_count: 1000000.
2021-07-19 13:46:22 [INFO] Data read start. data_count: 1000000.
2021-07-19 13:46:46 [INFO] Data read start. data_count: 1000000.
2021-07-19 13:47:07 [INFO] Data read start. data_count: 1000000.
2021-07-19 13:47:32 [INFO] Data read start. data_count: 1000000.
2021-07-19 13:47:56 [INFO] Data read start. data_count: 1000000.
2021-07-19 13:48:20 [INFO] Data read start. data_count: 1000000.
2021-07-19 13:48:44 [INFO] Data read start. data_count: 1000000.
2021-07-19 13:49:11 [INFO] Data read start. data_count: 1000000.
2021-07-19 13:49:35 [INFO] Data read start. data_count: 1000000.
2021-07-19 13:50:03 [INFO] Data read start. data_count: 1000000.
2021-07-19 13:50:27 [INFO] Data read start. data_count: 590838.
CPU times: user 2min 48s, 

In [10]:
len(shots_df)

13590838

In [11]:
# ショットメタデータ読み込み
shots_meta_df = dr.read_shots_meta(shots_meta_index)

## 全ショットから特徴点抽出
全ショットから最大荷重点、荷重開始点、破断点を抽出し、Elasticsearchに保存します。  

***target***: 対象ショットデータ（yyyyMMddHHMMSS）  
***shots_df***: ショットデータ  
***shots_meta_df***: ショットメタデータ  
***feature***: 抽出する特徴点名（max/start/break）  
***func***：適用するロジック  
***sub_func***：サブロジック  
***exclude_shots***：除外するショット番号（例：exclude_shots=(1227, 1228, 1229)）

保存先インデックスは以下になります。  
* 最大荷重点：shots-yyyyMMddHHMMSS-max-point
* 荷重開始点：shots-yyyyMMddHHMMSS-start-point
* 破断点：shots-yyyyMMddHHMMSS-break-point

In [12]:
# 不正ショットの除外
exclude_shots=(983, 1227, 1228, 1229, 1369, 1381, 2894)

In [13]:
analyzer = Analyzer(target, shots_df, shots_meta_df, exclude_shots)

In [14]:
# 最大荷重点
analyzer.apply(
    feature="max",
    func=ef.max_load,
    sub_func=None,
)

NameError: name 'ef' is not defined

In [ ]:
dr = DataReader()
max_index = "shots-" + target + "-max-point"
max_df = dr.read_all(max_index)
max_df

In [ ]:
# 荷重開始点
analyzer.apply(
    feature="start",
    func=load_start3,
    sub_func=None,
)

In [ ]:
dr = DataReader()
start_index = "shots-" + target + "-start-point"
start_df = dr.read_all(start_index)
start_df

In [ ]:
# 破断点
analyzer.apply(
    feature="break",
    func=breaking_vmin_amin,
    sub_func=narrowing_v4min_mab,
)

In [ ]:
dr = DataReader()
break_index = "shots-" + target + "-break-point"
break_df = dr.read_all(break_index)
break_df

In [ ]:
break_df.head(10)